In [13]:
import pandas as pd

In [1]:
import json

def get_content_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = json.load(file)
    return content

In [32]:
toc = get_content_from_json('..\\..\\database_export\\table_of_contents.json')

def get_something(title, something, toc):
    if type(toc) == list:
        for item in toc:
            g = get_something_for_me(title,something, item)
            if g is not None:
                return g
            
def get_something_for_me(title,something, toc):
    if 'title' in toc and toc['title'] == title and something in toc:
        return toc[something]
    if 'contents' in toc:
        for item in toc['contents']:
            g= get_something_for_me(title,something, item)
            if g is not None:
                return g
        
print(get_something('Genesis','heShortDesc', toc))

בריאת העולם, תחילתה של האנושות וסיפורי האבות והאמהות.


In [33]:
print(get_something('Devash LeFi','heShortDesc', toc))

החלק השני של קונטרס מדבר קדמות של החיד"א.


In [34]:

import os

json_folder = "..\\..\\database_export\\json"
schemas_folder = "..\\..\\database_export\\schemas"

data = []





with open("blacklist.txt", 'r', encoding='utf-8') as file:
    blacklist = file.read().splitlines()
    for root, _,files in os.walk(json_folder):
        for file in files:
            file_path = os.path.join(root, file)
            if  file_path.endswith('Hebrew\\merged.json') and file_path.split('\\')[-3] not in blacklist:
                title = file_path.split('\\')[-3].replace(' ', '_')
                schema_file_name = os.path.join(schemas_folder, title + '.json')
                schema = get_content_from_json(schema_file_name)
                heShortDesc = get_something(schema['title'], 'heShortDesc', toc)
                order = get_something(schema['title'], 'order', toc)
                heTitle = schema['heTitle'] if 'heTitle' in schema else ''
                pubPlace = schema['pubPlaceString']['he'] if 'pubPlaceString' in schema else ''
                pubData = schema['pubDateString']['he'] if 'pubDateString' in schema else ''
                compPlace = schema['compPlaceString']['he'] if 'compPlaceString' in schema else ''
                compDate = schema['compDateString']['he'] if 'compDateString' in schema else ''
                heDesc = schema['heDesc'] if 'heDesc' in schema else ''
                data.append({"title": heTitle, "PubDate": pubData,'pubPlace': pubPlace, 'compPlace': compPlace, 'compDate': compDate, 'heShortDesc': heShortDesc, 'order': order, 'heDesc': heDesc})

df = pd.DataFrame(data)
df
                   

,title,PubDate,pubPlace,compPlace,compDate,heShortDesc,order,heDesc
0,"חיי מוהר""ן",(1874 לספירה),למברג,,(1810 – 1830 לספירה בקירוב),ביוגרפיה של רבי נחמן שכתב תלמידו רבי נתן.,NaN,"תורות וקורות חייו של ר' נחמן מברסלב, נכתב ע""י ..."
1,ליקוטי עצות,(1816 לספירה),דובנו,,(1790 – 1810 לספירה בקירוב),אוסף דרשותיו של רבי נחמן העוסקות במידות טובות ...,NaN,
2,ליקוטי הלכות,(1843 – 1861 לספירה),,,(1814 – 1844 לספירה בקירוב),חיבורו רחב היריעה של רבי נתן המבאר את הלכות הש...,NaN,
3,"ליקוטי מוהר""ן",(1808 לספירה),אוסטראה,ברסלב,(1802 – 1808 לספירה בקירוב),"יצירת מופת של רבי נחמן, הכוללת פרשנות לאגדות ה...",NaN,
4,ליקוטי תפילות,(1821 לספירה),ורוצלב,ורוצלב,(1810 – 1820 לספירה בקירוב),״ליקוטי תפילות״ הוא אוסף מהמאה ה-19 הכולל 210 ...,NaN,״ליקוטי תפילות״ הוא אוסף 210 תפילות שחיבר רבי ...
...,...,...,...,...,...,...,...,...
5839,תוספתא מעשרות,(1521 לספירה),ונציה,ישראל התלמודית,(190 – 230 לספירה בקירוב),"הפרשת מעשרות מגידולי הקרקע לכהנים, ללווים ולענ...",7.0,
5840,תוספתא ערלה,(1521 לספירה),ונציה,ישראל התלמודית,(190 – 230 לספירה בקירוב),האיסור לאכול וליהנות מפירות העץ בשלוש השנים הר...,9.0,
5841,תוספתא פאה,(1521 לספירה),ונציה,ישראל התלמודית,(190 – 230 לספירה בקירוב),הקצאת חלק מתבואת השדה לעניים ומתנות אחרות מגיד...,2.0,
5842,תוספתא שביעית,(1521 לספירה),ונציה,ישראל התלמודית,(190 – 230 לספירה בקירוב),"דיני שנת השמיטה, השנה השביעית במחזור החקלאי, ש...",5.0,


In [35]:
df['heShortDesc'].unique()

array(['ביוגרפיה של רבי נחמן שכתב תלמידו רבי נתן.',
       'אוסף דרשותיו של רבי נחמן העוסקות במידות טובות ומידות רעות, מסודרות על פי סדר אלפביתי.',
       'חיבורו רחב היריעה של רבי נתן המבאר את הלכות השולחן ערוך לאור תורת חסידות ברסלב.',
       'יצירת מופת של רבי נחמן, הכוללת פרשנות לאגדות התלמוד ולחלקים מן הזוהר לצד דרשות נוספות.',
       '״ליקוטי תפילות״ הוא אוסף מהמאה ה-19 הכולל 210 תפילות אישיות וייחודיות שחיבר רבי נתן מנמירוב, תלמידו של רבי נחמן מברסלב, באוקראינה.',
       'אוסף דרשותיו של רבי נחמן על מידות טובות ומידות רעות, ערוך על פי סדר האלפבית.',
       'ביוגרפיה של רבי נחמן המתמקדת במסעו לארץ ישראל, שכתב תלמידו רבי נתן.',
       'לקט שיחות קצרות ותורות של רבי נחמן שהתפרסמו לראשונה כתוספת לחיבור סיפורי מעשיות.',
       'אוסף מעשיות שסיפר רבי נחמן, פורסם לראשונה בחיבור דו-שפתי (בעברית וביידיש).',
       'פירושים על תרי"ג מצוות ועל תפילות שכתב הצמח צדק, האדמו"ר השלישי של חסידות חב"ד.',
       '״ליקוטי תורה״ הוא ספר מרכזי בחסידות חב״ד שנכתב במאה ה-18 על ידי מייסדה - רבי שניאור ז

In [36]:
df.to_excel('metadata.xlsx',index=False)

In [20]:
! pip install pandas[excel]

     ---------------------------------------- 0.0/717.0 kB ? eta -:--:--
     - -------------------------------------- 20.5/717.0 kB ? eta -:--:--
     ---- ---------------------------------- 81.9/717.0 kB 1.5 MB/s eta 0:00:01
     ----------- -------------------------- 215.0/717.0 kB 1.6 MB/s eta 0:00:01
     ----------------- -------------------- 327.7/717.0 kB 1.8 MB/s eta 0:00:01
     ---------------------------- --------- 542.7/717.0 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 717.0/717.0 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   --------------